# TP 2: Modèles de Langage N-gram avec Tokenisation BPE

## Introduction

Dans ce TP, nous allons mettre en pratique les concepts de tokenisation et de modèles de langage n-gram vus en cours. Nous utiliserons un dataset de textes de Shakespeare pour:

1. Analyser les données textuelles
2. Implémenter et entraîner un tokenizer BPE (Byte Pair Encoding)
3. Construire un modèle de langage n-gram
4. Générer du texte à partir de notre modèle

Ce TP vous permettra de comprendre concrètement comment fonctionne la génération de texte à partir d'approches statistiques, avant d'aborder les modèles neuronaux plus complexes.

## Partie 1: Configuration et chargement des données

Commençons par importer les bibliothèques nécessaires :

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import Counter, defaultdict
import random
import requests
from typing import List, Dict, Tuple, Set
import torch
from tqdm.notebook import tqdm

Téléchargeons et chargeons notre corpus de Shakespeare :

In [ ]:
# URL du dataset Shakespeare
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

# Téléchargement du dataset
response = requests.get(url)
shakespeare_text = response.text

# Affichage d'un extrait
print(f"Longueur du texte: {len(shakespeare_text)} caractères")
print("\nExtrait du texte:")
print(shakespeare_text[:500])

## Partie 2: Analyse exploratoire des données

Analysons notre corpus pour mieux comprendre sa structure:

In [ ]:
# Nombre de caractères uniques
unique_chars = set(shakespeare_text)
print(f"Nombre de caractères uniques: {len(unique_chars)}")
print(f"Caractères uniques: {''.join(sorted(unique_chars))}")

# Fréquence des caractères
char_freq = Counter(shakespeare_text)
print("\nLes 10 caractères les plus fréquents:")
for char, count in char_freq.most_common(10):
    if char == '\n':
        char_display = '\\n'
    elif char == ' ':
        char_display = 'espace'
    else:
        char_display = char
    print(f"'{char_display}': {count} occurrences")

# Nombre de mots (approximatif)
words = re.findall(r'\b\w+\b', shakespeare_text.lower())
unique_words = set(words)
print(f"\nNombre total de mots: {len(words)}")
print(f"Nombre de mots uniques: {len(unique_words)}")
print("\nLes 10 mots les plus fréquents:")
word_freq = Counter(words)
for word, count in word_freq.most_common(10):
    print(f"'{word}': {count} occurrences")

## Partie 3: Implémentation du tokenizer BPE

Maintenant, implémentons notre propre tokenizer BPE (Byte Pair Encoding) pour comprendre comment il fonctionne. Comme vu en cours, BPE fonctionne en fusionnant progressivement les paires de tokens les plus fréquentes.

In [30]:
class BPETokenizer:
    def __init__(self):
        self.vocab = {}  # Vocabulaire: token -> id
        self.inverse_vocab = {}  # Id -> token
        self.merges = []  # Liste des fusions BPE
        self.char_level_tokens = True  # Indique si le tokenizer est au niveau des caractères
    
    def train(self, text: str, vocab_size: int, verbose: bool = True) -> List[Tuple[str, str]]:
        """
        Entraîne le tokenizer BPE sur un texte donné jusqu'à atteindre la taille 
        de vocabulaire souhaitée.
        
        Args:
            text: Texte d'entraînement
            vocab_size: Taille cible du vocabulaire
            verbose: Affiche des informations sur l'entraînement
            
        Returns:
            Liste des fusions effectuées
        """
        # Initialisation avec un vocabulaire de caractères
        unique_chars = sorted(set(text))
        self.vocab = {char: i for i, char in enumerate(unique_chars)}
        self.inverse_vocab = {i: char for i, char in enumerate(unique_chars)}
        
        # Tokenisation initiale du texte (chaque caractère est un token)
        tokens = list(text)
        
        # Statistiques pour visualisation
        vocab_sizes = [len(self.vocab)]
        token_counts = [len(tokens)]
        
        # Effectuer les fusions jusqu'à atteindre la taille du vocabulaire cible
        while len(self.vocab) < vocab_size:
            # Compter les paires de tokens adjacentes
            pairs = self._count_pairs(tokens)
            if not pairs:
                break
                
            # Trouver la paire la plus fréquente
            best_pair = max(pairs, key=pairs.get)
            
            # À COMPLÉTER: Fusionner la paire la plus fréquente dans le texte
            # ---------------------
            # Ajouter la nouvelle paire au vocabulaire
            new_token = ''.join(best_pair)
            self.vocab[new_token] = len(self.vocab)
            self.inverse_vocab[len(self.vocab) - 1] = new_token
            
            # Mettre à jour les tokens en fusionnant la paire sélectionnée
            tokens = self._merge_pair(tokens, best_pair, new_token)
            
            # Enregistrer cette fusion
            self.merges.append(best_pair)
            
            # ---------------------
            
            # Collecter des statistiques
            vocab_sizes.append(len(self.vocab))
            token_counts.append(len(tokens))
            
            if verbose and len(self.vocab) % 100 == 0:
                print(f"Vocabulaire: {len(self.vocab)}, Fusion: {best_pair} -> {new_token}")
            
            # Nous ne sommes plus au niveau des caractères
            self.char_level_tokens = False
            
        # Visualiser l'évolution du vocabulaire et du nombre de tokens
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(vocab_sizes)
        plt.xlabel('Nombre de fusions')
        plt.ylabel('Taille du vocabulaire')
        plt.title('Évolution de la taille du vocabulaire')
        
        plt.subplot(1, 2, 2)
        plt.plot(token_counts)
        plt.xlabel('Nombre de fusions')
        plt.ylabel('Nombre de tokens')
        plt.title('Évolution du nombre de tokens')
        plt.tight_layout()
        plt.show()
        
        return self.merges
    
    def _count_pairs(self, tokens: List[str]) -> Dict[Tuple[str, str], int]:
        """Compte les occurrences de chaque paire de tokens adjacents."""
        pairs = defaultdict(int)
        for i in range(len(tokens) - 1):
            pair = (tokens[i], tokens[i+1])
            pairs[pair] += 1
        return pairs
    
    def _merge_pair(self, tokens: List[str], pair: Tuple[str, str], new_token: str) -> List[str]:
        """Fusionne toutes les occurrences d'une paire dans une liste de tokens."""
        # À COMPLÉTER: Implémenter la fusion des paires
        # ---------------------
        i = 0
        result = []
        while i < len(tokens):
            # Si on trouve la paire à fusionner et qu'on n'est pas à la fin
            if i < len(tokens) - 1 and tokens[i] == pair[0] and tokens[i + 1] == pair[1]:
                result.append(new_token)
                i += 2
            else:
                result.append(tokens[i])
                i += 1
        return result
        # ---------------------
    
    def encode(self, text: str) -> List[int]:
        """
        Encode un texte en utilisant le tokenizer BPE entraîné.
        
        Args:
            text: Texte à encoder
            
        Returns:
            Liste des IDs de tokens
        """
        # Initialisation avec des caractères individuels
        tokens = list(text)
        
        # Appliquer les fusions dans l'ordre où elles ont été apprises
        for pair in self.merges:
            tokens = self._merge_pair(tokens, pair, ''.join(pair))
        
        # Convertir les tokens en IDs
        ids = [self.vocab.get(token, self.vocab.get("<unk>", 0)) for token in tokens]
        return ids
    
    def decode(self, ids: List[int]) -> str:
        """
        Décode une liste d'IDs de tokens en texte.
        
        Args:
            ids: Liste des IDs de tokens
            
        Returns:
            Texte décodé
        """
        # Convertir les IDs en tokens
        tokens = [self.inverse_vocab.get(id, "<unk>") for id in ids]
        
        # Concaténer les tokens
        text = ''.join(tokens)
        return text

Maintenant, entraînons notre tokenizer BPE sur le corpus de Shakespeare :

In [ ]:
# Entraînement du tokenizer BPE
tokenizer = BPETokenizer()
vocab_size = 1000  # Taille cible du vocabulaire

merges = tokenizer.train(shakespeare_text, vocab_size)

# Encodons un extrait pour tester
sample_text = shakespeare_text[:200]
encoded = tokenizer.encode(sample_text)
decoded = tokenizer.decode(encoded)

print(f"Texte original: {sample_text}")
print(f"Texte encodé (IDs): {encoded[:20]}...")
print(f"Texte décodé: {decoded}")
print(f"Taille du vocabulaire final: {len(tokenizer.vocab)}")
print(f"Nombre de tokens pour l'extrait: {len(encoded)}")
print(f"Nombre de caractères de l'extrait: {len(sample_text)}")

## Partie 4: Implémentation du modèle n-gram

Maintenant que nous avons notre tokenizer, implémentons un modèle de langage n-gram. Comme vu en cours, un modèle n-gram prédit le token suivant en se basant sur les n-1 tokens précédents.

In [32]:
class NgramLanguageModel:
    def __init__(self, n: int, tokenizer: BPETokenizer):
        """
        Initialise un modèle de langage n-gram
        
        Args:
            n: Ordre du modèle n-gram (nombre de tokens à considérer)
            tokenizer: Tokenizer BPE entraîné
        """
        self.n = n
        self.tokenizer = tokenizer
        self.vocab_size = len(tokenizer.vocab)
        self.context_counts = defaultdict(int)  # Compte des contextes (n-1 tokens)
        self.ngram_counts = defaultdict(int)    # Compte des n-grams complets
    
    def train(self, text: str):
        """
        Entraîne le modèle n-gram sur un texte
        
        Args:
            text: Texte d'entraînement
        """
        # Tokeniser le texte
        token_ids = self.tokenizer.encode(text)
        
        # À COMPLÉTER: Compter les n-grams et leurs contextes
        # ---------------------
        # Pour chaque position possible dans le texte tokenisé
        for i in range(len(token_ids) - self.n + 1):
            # Extraire le n-gram complet
            ngram = tuple(token_ids[i:i+self.n])
            
            # Extraire le contexte (n-1 premiers tokens du n-gram)
            context = ngram[:-1]
        # ---------------------
            
            # Incrémenter les compteurs
            self.context_counts[context] += 1
            self.ngram_counts[ngram] += 1
    
    def get_probability(self, context: Tuple[int], next_token: int) -> float:
        """
        Calcule la probabilité P(next_token | context)
        
        Args:
            context: Tuple des (n-1) token_ids précédents
            next_token: Token_id suivant à prédire
            
        Returns:
            Probabilité du token suivant étant donné le contexte
        """
        # À COMPLÉTER: Calculer la probabilité conditionnelle
        # ---------------------
        # Si le contexte n'a jamais été vu, retourner une distribution uniforme
        if context not in self.context_counts or self.context_counts[context] == 0:
            return 1.0 / self.vocab_size
        
        # Sinon, calculer la probabilité conditionnelle
        ngram = context + (next_token,)
        return self.ngram_counts[ngram] / self.context_counts[context]
        # ---------------------
    
    def generate_text(self, seed_text: str, length: int, temperature: float = 1.0) -> str:
        """
        Génère du texte à partir d'un texte d'amorce (seed)
        
        Args:
            seed_text: Texte d'amorce
            length: Nombre de tokens à générer
            temperature: Contrôle la randomisation (1.0 = fidèle aux probabilités,
                        < 1.0 = plus conservateur, > 1.0 = plus aléatoire)
                        
        Returns:
            Texte généré
        """
        # Encoder le texte d'amorce
        tokens = self.tokenizer.encode(seed_text)
        
        # S'assurer que nous avons au moins n-1 tokens
        if len(tokens) < self.n - 1:
            print(f"Avertissement: Le texte d'amorce est trop court. Ajout de tokens aléatoires.")
            tokens = [random.randint(0, self.vocab_size - 1) for _ in range(self.n - 1 - len(tokens))] + tokens
        
        # Ne garder que les n-1 derniers tokens comme contexte initial
        context = tuple(tokens[-(self.n - 1):])
        
        # Générer de nouveaux tokens
        generated_tokens = list(tokens)
        
        for _ in range(length):
            # Calculer les probabilités pour tous les tokens possibles suivant ce contexte
            probabilities = [self.get_probability(context, token_id) for token_id in range(self.vocab_size)]
            
            # Appliquer la température
            if temperature != 1.0:
                
                probabilities = [p ** (1.0 / temperature) for p in probabilities]
                # Normaliser pour s'assurer que la somme est égale à 1
                sum_probs = sum(probabilities)
                if sum_probs > 0:
                    probabilities = [p / sum_probs for p in probabilities]
            
            # Échantillonner le prochain token selon ces probabilités
            next_token = random.choices(range(self.vocab_size), weights=probabilities, k=1)[0]
            generated_tokens.append(next_token)
            
            # Mettre à jour le contexte
            context = context[1:] + (next_token,)
        
        # Décoder les tokens générés
        generated_text = self.tokenizer.decode(generated_tokens)
        return generated_text

## Partie 5: Entraînement et génération de texte

Maintenant, entraînons notre modèle n-gram et testons la génération de texte :

In [ ]:
# Définir l'ordre du modèle n-gram
n = 3  # Trigramme

# Créer et entraîner le modèle
ngram_model = NgramLanguageModel(n, tokenizer)

# Nous pouvons prendre un sous-ensemble du texte pour accélérer l'entraînement
training_text = shakespeare_text  # Premiers 100k caractères
print(f"Entraînement sur {len(training_text)} caractères...")

ngram_model.train(training_text)

# Analyser les statistiques du modèle
num_contexts = len(ngram_model.context_counts)
num_ngrams = len(ngram_model.ngram_counts)
print(f"Nombre de contextes uniques (n-1 grams): {num_contexts}")
print(f"Nombre de n-grams uniques: {num_ngrams}")

# Contexte le plus fréquent
if num_contexts > 0:
    most_common_context = max(ngram_model.context_counts.items(), key=lambda x: x[1])
    context_tokens = [tokenizer.inverse_vocab[token_id] for token_id in most_common_context[0]]
    print(f"Contexte le plus fréquent: {''.join(context_tokens)} (vu {most_common_context[1]} fois)")

# Générer du texte
seed_text = "Shall I compare thee to a summer's day?\n"
print(f"\nTexte d'amorce: {seed_text}")

# Générer avec différentes températures
for temp in [0.5, 1.0, 1.5]:
    generated_text = ngram_model.generate_text(seed_text, length=100, temperature=temp)
    print(f"\nTexte généré (température={temp}):\n{generated_text}")

In [ ]:
for ngram_count in sorted(ngram_model.ngram_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"tokens: {ngram_count[0]}, decoded: {tokenizer.decode(ngram_count[0])}, count: {ngram_count[1]}")


## Partie 6: Expérimentations

Maintenant que nous avons notre infrastructure en place, expérimentons avec différents paramètres :

In [ ]:
# 1. Essayez différentes tailles de vocabulaire pour le tokenizer BPE
# 2. Essayez différents ordres de n-grams (n=2, n=3, n=4, etc.)
# 3. Essayez différentes températures pour la génération
# 4. Comparez les résultats et analysez les différences

# Exemple:
# Essayons avec un modèle bigram (n=2)
bigram_model = NgramLanguageModel(2, tokenizer)
bigram_model.train(training_text)
print("\nGénération avec un modèle bigram:")
bigram_text = bigram_model.generate_text(seed_text, length=100, temperature=1.0)
print(bigram_text)

# Essayons avec un modèle 4-gram
fourgram_model = NgramLanguageModel(4, tokenizer)
fourgram_model.train(training_text)
print("\nGénération avec un modèle 4-gram:")
fourgram_text = fourgram_model.generate_text(seed_text, length=100, temperature=1.0)
# Analyser les statistiques du modèle


In [ ]:
num_contexts = len(fourgram_model.context_counts)
num_ngrams = len(fourgram_model.ngram_counts)
print(f"Nombre de contextes uniques (n-1 grams): {num_contexts}")
print(f"Nombre de n-grams uniques: {num_ngrams}")
print(fourgram_text)

## Partie 7: Questions de réflexion

À la fin de ce TP, réfléchissez aux questions suivantes :

1. Comment la taille du vocabulaire BPE affecte-t-elle la qualité du texte généré?

2. Quelle est l'influence de l'ordre n du modèle n-gram sur:
   - La qualité du texte généré
   - La capacité du modèle à capturer le style de Shakespeare
   - La diversité du texte généré

3. Quelles sont les limites principales des modèles n-gram pour la génération de texte?

4. Comment pourriez-vous améliorer le modèle n-gram (techniques de lissage, backoff, etc.)?

5. Comparez cette approche avec les techniques modernes basées sur les réseaux de neurones. Quels avantages les modèles comme GPT pourraient-ils avoir sur cette approche?

## Conclusion

Dans ce TP, vous avez mis en pratique les concepts de tokenisation BPE et de modèles n-gram vus en cours. Vous avez implémenté ces algorithmes depuis zéro et les avez utilisés pour générer du texte dans le style de Shakespeare.

Ces techniques constituent la base historique de la génération de texte, et bien qu'elles aient été largement dépassées par les modèles neuronaux modernes, elles restent importantes pour comprendre les fondements du traitement automatique du langage.

Dans le prochain TP, nous explorerons des approches plus modernes basées sur les réseaux de neurones pour la génération de texte. 